## Import library

In [303]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)

## Functions

In [ ]:
def preprocess_raw_file(dataframe):
    dataframe['Month'] = dataframe['UPC'][dataframe['UPC'].apply(lambda x: isinstance(x, str))]
    dataframe['UPC'] = pd.to_numeric(dataframe['UPC'], errors = 'coerce')
    dataframe['Month'] = pd.to_datetime(dataframe['Month'], errors = 'coerce')
    dataframe['Month'] = dataframe['Month'].fillna(method = 'ffill')
    dataframe = dataframe[dataframe['UPC'].notnull()]
    
    cols_to_align = ['UPC', 'Purch QTY', 'QTY', 'QTY, On Hand']
    for col in cols_to_align:
        dataframe[col] = dataframe[col].apply(lambda x: int(float(x)))

    dataframe['Month'] = dataframe['Month'].dt.to_period('M')

    return dataframe

In [311]:
def get_vendor_pivot_table(dataframe, vendor_name):
    df_vendor = dataframe[dataframe['Vendor'] == vendor_name]
    # unique_months = df_vendor['Month'].sort_values().unique()
    min_month = df_vendor['Month'].min()
    max_month = df_vendor['Month'].max()
    unique_months = pd.date_range(min_month.to_timestamp().date(), max_month.to_timestamp().date(), freq='MS').to_period('M')
    
    df_pivot = pd.pivot_table(data = df_vendor, columns = 'Month', index = "Item Name", values = ['Purch QTY', 'QTY', 'QTY, On Hand'], \
                          aggfunc = np.sum)
    df_pivot = df_pivot.swaplevel(axis=1).sort_index(axis=1, level=0)

    # insert missing months
    column_order = ['Purch QTY', 'QTY', 'QTY, On Hand']
    new_columns = []
    for month in unique_months:
        for col_name in column_order:
            # if (month, col_name) in df_pivot.columns:
            new_columns.append((month, col_name))
    # df_pivot = df_pivot[new_columns]  # Reassign with ordered columns
    df_pivot = df_pivot.reindex(columns=new_columns)
    df_pivot = df_pivot.fillna(0)
    # df_pivot = df_pivot.fillna(0)
    return df_pivot, unique_months

In [312]:
def update_on_hand_qty(df_pivot, unique_months):
    for current_month in unique_months:
        if current_month == unique_months.min():
            df_pivot[(current_month, 'QTY, On Hand')] = df_pivot[(current_month, 'Purch QTY')] - df_pivot[(current_month, 'QTY')]
    
        else:
            previous_month = current_month - 1
            df_pivot[(current_month, 'QTY, On Hand')] = df_pivot[(previous_month, 'QTY, On Hand')] +\
            df_pivot[(current_month, 'Purch QTY')] - df_pivot[(current_month, 'QTY')]

    return df_pivot

In [313]:
def add_liq_percent(df_pivot, unique_months):

    for current_month in unique_months:
        qty = df_pivot[(current_month, 'QTY')]
        on_hand = df_pivot[(current_month, 'QTY, On Hand')]
        
        with np.errstate(divide='ignore', invalid='ignore'):
            liq_percent = qty / (qty + on_hand)
            liq_percent[np.isinf(liq_percent)] = 0
            liq_percent = np.nan_to_num(liq_percent, nan=0)  # handle NaNs if needed
    
        df_pivot[(current_month, 'LIQ %')] = liq_percent
        column_order = ['Purch QTY', 'QTY', 'QTY, On Hand', 'LIQ %']

        new_columns = []
        for month in unique_months:
            for col_name in column_order:
                if (month, col_name) in df_pivot.columns:
                    new_columns.append((month, col_name))
        df_pivot = df_pivot[new_columns]  # Reassign with ordered columns
        
    return df_pivot

In [314]:
def add_sale_qty(df_pivot, unique_months):

    for current_month in unique_months:
        if current_month == unique_months.min():
            df_pivot['Sale QTY'] = df_pivot[(current_month, 'QTY')]
        else:
            df_pivot['Sale QTY'] += df_pivot[(current_month, 'QTY')]
    return df_pivot

In [315]:
def add_shelf_life(df_pivot, unique_months):

    df_pivot['Shelf Life'] = 0
    for current_month in unique_months:
        df_pivot.loc[df_pivot[(current_month, 'QTY')] > 0, 'Shelf Life'] += 1
    return df_pivot

In [316]:
def add_avg_sales_per_month(df_pivot):

    df_pivot['Avg Sales/Mnth'] = df_pivot['Sale QTY'] / df_pivot['Shelf Life']
    df_pivot['Avg Sales/Mnth'] = df_pivot['Avg Sales/Mnth'].fillna(0)
    return df_pivot

In [317]:
def mround_excel(number, multiple):
    import math
    return multiple * round((number + 1e-10) / multiple) if number >= 0 else multiple * round((number - 1e-10) / multiple)

In [318]:
def add_bi_weekly_sale_forecasting(df_pivot):

    df_pivot['Bi-Weekly Sale Fsct'] = df_pivot['Avg Sales/Mnth'].apply(lambda x: mround_excel(x / 2, 1))

    return df_pivot

In [319]:
def add_safety_stock(df_pivot):

    df_pivot['Safety STK'] = df_pivot['Bi-Weekly Sale Fsct'].apply(lambda x: mround_excel(x * 0.5, 1))
    return df_pivot

In [320]:
def add_order(df_pivot, latest_month):
    df_pivot.loc[df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] > df_pivot[(latest_month, 'QTY, On Hand')], 'Order'] = \
    df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] - df_pivot[(latest_month, 'QTY, On Hand')]
    df_pivot.loc[df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] <= df_pivot[(latest_month, 'QTY, On Hand')], 'Order'] = 0
    return df_pivot

## Import file and preprocess

In [321]:
df = pd.read_excel('./src/doc_formatter/assets/data/example.xlsx', engine = 'openpyxl')

In [322]:
df.columns = df.iloc[7]

In [323]:
df.columns.name = None

In [324]:
df = df.loc[8:].reset_index(drop = True)

In [325]:
df_processed = preprocess_raw_file(df)

In [326]:
df_processed

,UPC,Item Name,GTIN,Size,Purch QTY,"Purch Amount,$",QTY,"Cost,$","QTY, On Hand","Cost, On Hand,$","Retail, On Hand,$","Buydown by Purch,$",Vendor,Last Purch date,VIN,"Retail,$","Unit Cost,$","Unit Retail,$","Profit,$",GPM%,"Non-itemized Sales,$","Buydown by Sales,$","Total,$",Month
1,69158402237,Stlth Type-C Black Metal,00691584022375,EA,5,37.45,0,0,0,0,0,0,Smoke Arsenal,08.30.2024,29990000001,0,0,0,0,0,0,0,0,2024-03
2,69158402243,Stlth Type-C Device Green Metal,00691584022436,EA,5,37.45,0,0,0,0,0,0,Smoke Arsenal,03.04.2024,29990000001,0,0,0,0,0,0,0,0,2024-03
3,505634808873,IVG Punch,05056348088731,3000 Puffs,0,0,2,10,0,0,0,0,Smoke Arsenal,06.11.2024,29990000001,40,5,20,30,75,0,0,40,2024-03
4,505661751113,IVG Mango Ice,05056617511137,3000 Puffs,0,0,2,10,0,0,0,0,Smoke Arsenal,06.11.2024,29990000001,40,5,20,30,75,0,0,40,2024-03
5,505661751203,IVG Max Mango Peach Watermelon,05056617512035,5000 Puffs,0,0,5,84.95,0,0,0,0,Smoke Arsenal,10.17.2024,MMPW1020FDR,125,16.99,25,40.05,32.04,0,0,125,2024-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2889,697745490325,Geek Bar Pulse X Peach Lemon Ice,06977454903253,25000 Puffs,4,84.88,4,84.88,0,0,0,0,Maq Distributors,06.13.2025,gbv1317b-fd,168,21.22,42,83.12,49.48,0,0,168,2025-06
2890,697745490329,Geek Bar Pulse X Apple Kiwi Ice,06977454903291,25000 Puffs,4,84.88,4,84.88,0,0,0,0,Maq Distributors,06.13.2025,gbv1312b-fd,168,21.22,42,83.12,49.48,0,0,168,2025-06
2891,697745490331,Geek Bar Pulse X Razz Mango Ice,06977454903314,25000 Puffs,4,84.88,4,84.88,0,0,0,0,Maq Distributors,06.13.2025,gbv1319b-fd,168,21.22,42,83.12,49.48,0,0,168,2025-06
2892,697745490333,Geek Bar Pulse X Strawnana Orange Ice,06977454903338,25000 Puffs,4,84.88,4,84.88,0,0,0,0,Maq Distributors,06.13.2025,gbv1321b-fd,168,21.22,42,83.12,49.48,0,0,168,2025-06


## Filter Vendor

In [327]:
# Vapes
# 1. Rankin
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
# 2. Walpole
    # a. Smoke Tokes
    # b. Maq Distribution
 
# 3. Fort Frances
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
# 4. Silver Grizzly
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
 
# Cannabis
# 1. Silver Grizzly

    # a. Proulx Commercial Growers
    # b. Cannabis Soda Co.
    # c. Native Flower

In [328]:
df_processed['Vendor'].unique()

array(['Smoke Arsenal', nan, 'Fog Formulas', 'Smoke Tokes',
       'G-Core Canada', 'Maq Distributors', 'Core-Mark'], dtype=object)

In [19]:
target_vendor = 'G-Core Canada'

In [259]:
target_vendor = 'Core-Mark'

In [329]:
target_vendor = 'Maq Distributors'

## Create Pivot table

In [330]:
df_pivot, unique_months = get_vendor_pivot_table(df_processed, target_vendor)

### Update QTY, On Hand

In [331]:
df_pivot = update_on_hand_qty(df_pivot, unique_months)

### Add LIQ %

In [332]:
df_pivot = add_liq_percent(df_pivot, unique_months)

### Add Sale QTY

In [333]:
df_pivot = add_sale_qty(df_pivot, unique_months)

### Add Shelf Life

In [334]:
df_pivot = add_shelf_life(df_pivot, unique_months)

### Add Avg Sales/Mnth

In [335]:
df_pivot = add_avg_sales_per_month(df_pivot)

### Add Bi-Weekly Sale Fsct

In [336]:
df_pivot = add_bi_weekly_sale_forecasting(df_pivot)

### Add Safety STK

In [337]:
df_pivot = add_safety_stock(df_pivot)

### Order

In [338]:
latest_month = unique_months.max()

In [339]:
df_pivot = add_order(df_pivot, latest_month)

In [340]:
# sorting
df_pivot = df_pivot.sort_values(by = 'Avg Sales/Mnth', ascending = False)

In [341]:
# # export
# df_pivot.to_csv('df_pivot.csv')

In [342]:
df_pivot.to_excel('output.xlsx', sheet_name='document_formatted')

In [278]:
df_processed[df_processed['Vendor'] == target_vendor]['Item Name'].nunique()

1